In [1]:
from pyspark.sql import *
import plotly.plotly as py
#from pyspark.mllib.stat import Statistics
#import numpy as np
py.sign_in('harshitshah', 'rdckn6jnj9')
from plotly.graph_objs import *
import pandas as pd
import requests
import matplotlib
from pyspark.ml.feature import Normalizer, StandardScaler
from pyspark.mllib.classification import SVMWithSGD, SVMModel, LogisticRegressionWithLBFGS
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.regression import LinearRegressionWithSGD, RidgeRegressionWithSGD

#read data into pandas dataframe
df = pd.read_csv('YearPredictionMSD.txt')
df.columns = ['year','c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','c11','c12','c13','c14','c15','c16','c17','c18','c19','c20','c21','c22','c23','c24','c25','c26','c27','c28','c29','c30','c31','c32','c33','c34','c35','c36','c37','c38','c39','c40','c41','c42','c43','c44','c45','c46','c47','c48','c49','c50','c51','c52','c53','c54','c55','c56','c57','c58','c59','c60','c61','c62','c63','c64','c65','c66','c67','c68','c69','c70','c71','c72','c73','c74','c75','c76','c77','c78','c79','c80','c81','c82','c83','c84','c85','c86','c87','c88','c89','c90']

#Statistics
df_sample = df.sample(100)
print df.describe()

#histogram and box-plot for year
x0 = df['year']
trace = Data([Histogram(x=x0, histnorm='count', name='Year')])
trace0 = Box(x=x0, , name='Year_Boxplot')
data = Data([trace0])
data1 = Data([trace])
plot_url2 = py.plot(data1, filename='basic-histogram')
plot_url = py.plot(data, filename='basic-box-plot')


#normalize
df_norm = (df - df.mean())/df.std()

#Split in test and train
df_train = df_norm[:463715]
df_test = df_norm[463716:515345]

#write dataframe to csv
df_train.to_csv('df_train_Q1.csv',header = False)
df_test.to_csv('df_test_Q1.csv',header = False)

#defining a function to split values
def parsePoint(line):
    values = [float(x) for x in line.split(',')]
    return LabeledPoint(values[1], values[2:91])

#reading csv file into rdd
rdd_train = sc.textFile("df_train_Q1.csv")
rdd_test = sc.textFile("df_test_Q1.csv")

#parsing data
parsedData_train = rdd_train.map(parsePoint)
parsedData_test = rdd_test.map(parsePoint)

#run LinearRegression
model = LinearRegressionWithSGD.train(parsedData_train)

#Evaluate the model on training data
valuesAndPreds_train = parsedData_train.map(lambda p: (p.label, model.predict(p.features)))
MSE = valuesAndPreds_train.map(lambda (v, p): (v - p)**2).reduce(lambda x, y: x + y) / valuesAndPreds_train.count()
print("Training Mean Squared Error for Linear Regression = " + str(MSE))

#Evaluate the model on testing data
valuesAndPreds_test = parsedData_test.map(lambda p: (p.label, model.predict(p.features)))
MSE = valuesAndPreds_test.map(lambda (v, p): (v - p)**2).reduce(lambda x, y: x + y) / valuesAndPreds_test.count()
print("Testing Mean Squared Error for Linear Regression = " + str(MSE))


#run RidgeRegression
model = RidgeRegressionWithSGD.train(parsedData_train)

#Evaluate the model on training data
valuesAndPreds_train1 = parsedData_train.map(lambda p: (p.label, model.predict(p.features)))
MSE = valuesAndPreds_train1.map(lambda (v, p): (v - p)**2).reduce(lambda x, y: x + y) / valuesAndPreds_train1.count()
print("Training Mean Squared Error for Ridge Regression = " + str(MSE))

#Evaluate the model on testing data
valuesAndPreds_test1 = parsedData_test.map(lambda p: (p.label, model.predict(p.features)))
MSE = valuesAndPreds_test1.map(lambda (v, p): (v - p)**2).reduce(lambda x, y: x + y) / valuesAndPreds_test1.count()
print("Testing Mean Squared Error for Ridge Regression = " + str(MSE))

                year             c1             c2             c3  \
count  515344.000000  515344.000000  515344.000000  515344.000000   
mean     1998.397077      43.387113       1.289515       8.658222   
std        10.931056       6.067557      51.580393      35.268505   
min      1922.000000       1.749000    -337.092500    -301.005060   
25%      1994.000000      39.954667     -26.059848     -11.462775   
50%      2002.000000      44.258490       8.417725      10.476235   
75%      2006.000000      47.833875      36.124030      29.764685   
max      2011.000000      61.970140     384.065730     322.851430   

                  c4             c5             c6             c7  \
count  515344.000000  515344.000000  515344.000000  515344.000000   
mean        1.164110      -6.553580      -9.521968      -2.391046   
std        16.322802      22.860803      12.857763      14.571853   
min      -154.183580    -181.953370     -81.794290    -188.214000   
25%        -8.487507     -20.6664

In [15]:
from pyspark.sql import *
import plotly.plotly as py
#from pyspark.mllib.stat import Statistics
#import numpy as np
py.sign_in('harshitshah', 'rdckn6jnj9')
from plotly.graph_objs import *
import pandas as pd
import requests
from matplotlib import *
import matplotlib.pyplot as plt
from pyspark.ml.feature import Normalizer, StandardScaler
from pyspark.mllib.classification import SVMWithSGD, SVMModel, LogisticRegressionWithLBFGS
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.regression import LinearRegressionWithSGD, RidgeRegressionWithSGD

#read data into pandas dataframe
df = pd.read_csv('YearPredictionMSD.txt')
df.columns = ['year','c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','c11','c12','c13','c14','c15','c16','c17','c18','c19','c20','c21','c22','c23','c24','c25','c26','c27','c28','c29','c30','c31','c32','c33','c34','c35','c36','c37','c38','c39','c40','c41','c42','c43','c44','c45','c46','c47','c48','c49','c50','c51','c52','c53','c54','c55','c56','c57','c58','c59','c60','c61','c62','c63','c64','c65','c66','c67','c68','c69','c70','c71','c72','c73','c74','c75','c76','c77','c78','c79','c80','c81','c82','c83','c84','c85','c86','c87','c88','c89','c90']

#histogram for age, race and education
x0 = df['year']

#trace = Data([Histogram(x=x0, histnorm='count', name='Year')])
trace0 = Box(x=x0)
data = Data([trace0])

#plot_url2 = py.plot(trace, filename='basic-histogram')
plot_url = py.plot(data, filename='basic-box-plot')

/Library/Python/2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.

